In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR, SequentialLR, LambdaLR, OneCycleLR
import os
import matplotlib.pyplot as plt

In [2]:
lambda_data = '/mnt/d/Users/Admin/Projects/Machine_Learning/data'
delta_data = '/workspace/alvin/Machine_Learning_Studying/data'
data_lst = [lambda_data, delta_data]
data_pth = data_lst[1]

In [3]:
data_transforms = {
    "train" : transforms.Compose([
        transforms.Resize(size = (224, 224)), # resize to 224 by 224 for resnet
        # transforms.CenterCrop(size = (224, 224)),
        transforms.Grayscale(num_output_channels = 3), # this converts grayscale to rgb channels
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ]),
    "test" : transforms.Compose([
        transforms.Resize(size = (224, 224)), # resize to 224 by 224 for resnet
        # transforms.CenterCrop(size = (224, 224)),
        transforms.Grayscale(num_output_channels = 3), # this converts grayscale to rgb channels
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
}

In [4]:
labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

In [5]:
training_data = datasets.FashionMNIST(
    root = data_pth,
    train = True,
    download = True,
    transform = data_transforms["train"]
)

train_ds, val_ds = random_split(training_data, [50_000, 10_000])

ds_dict = {"train" : train_ds, "val" : val_ds}
dataset_sizes = {"train" : len(train_ds), "val" : len(val_ds)}

In [6]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /workspace/alvin/Machine_Learning_Studying/data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=3)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [11]:
dataloaders = {x : DataLoader(ds_dict[x], batch_size = 256, num_workers = 1, shuffle = True) for x in ds_dict.keys()}

In [12]:
# load pre-trained model
model = models.resnet18(weights = "DEFAULT")
# Replace final layer for the number of classes
model.fc = nn.Linear(model.fc.in_features, len(labels_map))

# Freeze all layers except final layer
# final layer is responsible for classification
for name, param in model.named_parameters():
    if "fc" in name:
        # unfreeze the fc layers
        # this means we are only training the fc layers
        param.requires_grad = True 
    else:
        param.requires_grad = False

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss() # most common used nn for classification problems
# optimizer = optim.SGD(model.parameters(), lr = 0.01)
# scheduler = StepLR(optimizer, step_size = 10, gamma = 0.1) # this multiplies lr every 10 epoch by 0.1

optimizer = optim.SGD(model.parameters(), lr = 0.001)

# def warmup_lambda(epoch):
#     if epoch < 20:
#         # scale from 0.5 (0.1/0.2) up to 1.0
#         return 0.5 + (epoch / 20) * 0.5
#     return 1.0

# scheduler1 = LambdaLR(optimizer, lr_lambda = warmup_lambda)
# scheduler2 = StepLR(optimizer, step_size = 4, gamma = 0.5)
# scheduler = SequentialLR(optimizer, schedulers = [scheduler1, scheduler2], milestones = [20])

scheduler = OneCycleLR(optimizer, max_lr = 0.0125, epochs = 50, steps_per_epoch = len(dataloaders["train"]))

# move model to GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [13]:
# Training loops
num_epochs = 50
for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    for phase in ["train", "val"]:
        if phase == "train":
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0 # correct predictions

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # clear the gradient from previous iteration

            with torch.set_grad_enabled(phase == "train"):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels) # check if output and labels match

                if phase == "train":
                    loss.backward()
                    optimizer.step()
                    scheduler.step() # scheduler here if OneCycleLR

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # scheduler.step()
    print(f"Epoch {epoch+1} LR: {scheduler.get_last_lr()[0]:.10f}")

print("Training complete!")

Epoch 0


KeyboardInterrupt: 

In [10]:
# Save model
torch.save(model.state_dict(), "/mnt/d/Users/Admin/Projects/Machine_Learning/weights/fashionmnist_epoch_50_resnet18_adaptive_lambda.pth")

## Classification on Test data

In [11]:
new_model = models.resnet18(weights = None) # dont load ImageNet Weights
new_model.fc = nn.Linear(new_model.fc.in_features, len(labels_map))

# Load your trained weights
new_model.load_state_dict(torch.load(
    "/mnt/d/Users/Admin/Projects/Machine_Learning/weights/fashionmnist_epoch_50_resnet18_adaptive_lambda.pth",
    map_location=device
))

new_model = new_model.to(device)
new_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
test_data = datasets.FashionMNIST(
    root = data_pth,
    train = False,
    download = True,
    transform = data_transforms["test"]
)

test_loader = DataLoader(
    test_data,
    batch_size = 128,
    num_workers = 8,
    shuffle = False
)

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = new_model(inputs)
        _, preds = torch.max(outputs, 1)

        correct += torch.sum(preds == labels).item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.8669


In [14]:
for i in range(100):
    image, label = test_data[i]
    image_batch = image.unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = new_model(image_batch)
        _, preds = torch.max(outputs, 1)

    pred_class = preds.item()
    
    # plt.imshow(image[0].squeeze(), cmap = "gray")
    # plt.title(f"image {i+1}")
    print(f"image {i+1} \n Pred: {labels_map[pred_class]}, True: {labels_map[label]}")
    plt.show()

image 1 
 Pred: Ankle Boot, True: Ankle Boot
image 2 
 Pred: Pullover, True: Pullover
image 3 
 Pred: Trouser, True: Trouser
image 4 
 Pred: Trouser, True: Trouser
image 5 
 Pred: Shirt, True: Shirt
image 6 
 Pred: Trouser, True: Trouser
image 7 
 Pred: Coat, True: Coat
image 8 
 Pred: Shirt, True: Shirt
image 9 
 Pred: Sandal, True: Sandal
image 10 
 Pred: Sneaker, True: Sneaker
image 11 
 Pred: Coat, True: Coat
image 12 
 Pred: Sandal, True: Sandal
image 13 
 Pred: Sneaker, True: Sneaker
image 14 
 Pred: Dress, True: Dress
image 15 
 Pred: Coat, True: Coat
image 16 
 Pred: Trouser, True: Trouser
image 17 
 Pred: Pullover, True: Pullover
image 18 
 Pred: T-Shirt, True: Coat
image 19 
 Pred: Bag, True: Bag
image 20 
 Pred: T-Shirt, True: T-Shirt
image 21 
 Pred: Pullover, True: Pullover
image 22 
 Pred: Sneaker, True: Sandal
image 23 
 Pred: Sneaker, True: Sneaker
image 24 
 Pred: Sandal, True: Ankle Boot
image 25 
 Pred: Trouser, True: Trouser
image 26 
 Pred: Shirt, True: Coat
image 